# Imports

In [37]:
import numpy as np
import pandas as pd

# Get datasets

In [8]:
# Download data from previous tests from Maritaca website
# If its the first time running, this will create the folder for you

# Download data from 2022 into folder datasets
!wget -cP ./datasets https://huggingface.co/datasets/maritaca-ai/enem/resolve/main/2022.jsonl
# Download data from 2023 into folder datasets
!wget -cP ./datasets https://huggingface.co/datasets/maritaca-ai/enem/resolve/main/2023.jsonl

--2024-11-20 17:48:12--  https://huggingface.co/datasets/maritaca-ai/enem/resolve/main/2022.jsonl
Resolving huggingface.co (huggingface.co)... 108.158.173.37, 108.158.173.9, 108.158.173.50, ...
Connecting to huggingface.co (huggingface.co)|108.158.173.37|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2024-11-20 17:48:13--  https://huggingface.co/datasets/maritaca-ai/enem/resolve/main/2023.jsonl
Resolving huggingface.co (huggingface.co)... 108.158.173.37, 108.158.173.9, 108.158.173.50, ...
Connecting to huggingface.co (huggingface.co)|108.158.173.37|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
# Download microdata from Enem website 2023 and unzip
!wget -cP ./datasets https://download.inep.gov.br/microdados/microdados_enem_2023.zip
!unzip datasets/microdados_enem_2023.zip -d ./datasets/2023

# Download microdata from Enem website 2022 and unzip
# !wget -cP ./datasetshttps://download.inep.gov.br/microdados/microdados_enem_2022.zip
!unzip datasets/microdados_enem_2022.zip -d ./datasets/2022

Archive:  datasets/microdados_enem_2022.zip
   creating: ./datasets/2022/LEIA-ME E DOCUMENTOS TÉCNICOS/
  inflating: ./datasets/2022/LEIA-ME E DOCUMENTOS TÉCNICOS/Edital_Enem_2022_Digital.pdf  
  inflating: ./datasets/2022/LEIA-ME E DOCUMENTOS TÉCNICOS/Edital_Enem_2022_Impresso.pdf  
  inflating: ./datasets/2022/LEIA-ME E DOCUMENTOS TÉCNICOS/enem_procedimentos_de_analise.pdf  
  inflating: ./datasets/2022/LEIA-ME E DOCUMENTOS TÉCNICOS/entenda_a_sua_nota_no_enem_guia_do_participante (2).pdf  
  inflating: ./datasets/2022/LEIA-ME E DOCUMENTOS TÉCNICOS/Leia_Me_Enem_2022.pdf  
  inflating: ./datasets/2022/LEIA-ME E DOCUMENTOS TÉCNICOS/manual_de_redacao_do_enem_2020.pdf  
  inflating: ./datasets/2022/LEIA-ME E DOCUMENTOS TÉCNICOS/matriz_referencia_enem.pdf  
   creating: ./datasets/2022/PROVAS E GABARITOS/
  inflating: ./datasets/2022/PROVAS E GABARITOS/ENEM_2022_DIGITAL_CAD_01_DIA_1_AZUL_ESPANHOL.pdf  
  inflating: ./datasets/2022/PROVAS E GABARITOS/ENEM_2022_DIGITAL_CAD_01_DIA_1_AZUL_INGL

# Clean dataset

## Load questions

In [74]:
# Load data from 2022 and 2023 then concat into a unique dataframe
df_questions_2022 = pd.read_json("./datasets/2022.jsonl", lines=True)
df_questions_2023 = pd.read_json("./datasets/2023.jsonl", lines=True)
df_questions = pd.concat([df_questions_2022, df_questions_2023], axis=0)
df_questions.head()

,id,exam,IU,ledor,question,description,alternatives,label,figures
0,questao_01,2022,True,True,"[[placeholder]]\nNessa tirinha, o comportament...",[Descrição da imagem: Tirinha apresentada em q...,"[revolta com a falta de sorte., gosto pela prá...",B,[https://raw.githubusercontent.com/piresramon/...
1,questao_02,2022,False,True,## A Teen’s View of Social Media\nInstagram is...,[],"[oferecer recursos de fotografia., divulgar pr...",D,[]
2,questao_03,2022,False,True,I tend the mobile now like an injured bird\nWe...,[],"[contentamento com a interação virtual., zelo ...",E,[]
3,questao_04,2022,False,True,"Two hundred years ago, Jane Austen lived in a ...",[],[problematizar o papel de gênero em casamentos...,C,[]
4,questao_05,2022,False,True,"As my official bio reads, I was made in Cuba, ...",[],"[qualidade da educação formal em Miami., prest...",D,[]


In [75]:
# Get all data types
df_questions.dtypes

id              object
exam             int64
IU                bool
ledor             bool
question        object
description     object
alternatives    object
label           object
figures         object
dtype: object

In [76]:
# Get statistical data from numeric values
df_questions.describe()


,exam
count,360.000000
mean,2022.500000
std,0.500696
min,2022.000000
25%,2022.000000
50%,2022.500000
75%,2023.000000
max,2023.000000


In [ ]:
# Create a boolean column to indicate if it has figures or not
df_questions['has_figures'] = np.where( df_questions['figures'].str.len() > 0, True, False)

In [112]:
# Drop unecessary columns
df_questions.drop(['IU', 'ledor', 'figures'], axis=1)

,id,exam,question,description,alternatives,label,has_figures
0,questao_01,2022,"[[placeholder]]\nNessa tirinha, o comportament...",[Descrição da imagem: Tirinha apresentada em q...,"[revolta com a falta de sorte., gosto pela prá...",B,True
1,questao_02,2022,## A Teen’s View of Social Media\nInstagram is...,[],"[oferecer recursos de fotografia., divulgar pr...",D,False
2,questao_03,2022,I tend the mobile now like an injured bird\nWe...,[],"[contentamento com a interação virtual., zelo ...",E,False
3,questao_04,2022,"Two hundred years ago, Jane Austen lived in a ...",[],[problematizar o papel de gênero em casamentos...,C,False
4,questao_05,2022,"As my official bio reads, I was made in Cuba, ...",[],"[qualidade da educação formal em Miami., prest...",D,False
...,...,...,...,...,...,...,...
175,questao_176,2023,O mastro de uma bandeira foi instalado perpend...,[Descrição da figura: A figura representa o ma...,"[22 x sqrt(3)/3., 11 x sqrt(2)., 12 x sqrt(2)....",C,True
176,questao_177,2023,Um controlador de voo dispõe de um instrumento...,"[Descrição do gráfico: Gráfico cartesiano, em ...","[1 : 5., 1 : 11., 1 : 55., 1 : 5 000., 1 : 500...",E,True
177,questao_178,2023,O calendário maia apresenta duas contagens sim...,[],"[741, 1 040, 1 460, 2 100, 5 200]",C,False
178,questao_179,2023,"Sejam a, b e c as medidas dos lados de um triâ...",[Descrição da figura: A figura apresenta um tr...,"[0 grau < alfa < 90 graus, alfa = 90 graus., 9...",C,True


## Load Enem Microdata